In [1]:
import pandas as pd
import numpy as np
from valuation.asset.identity.dataset import DatasetID
from valuation.core.stage import DatasetStage
from valuation.infra.store.dataset import DatasetStore


## Get Training Data

In [2]:
store = DatasetStore()
dataset_id = DatasetID(name="train_val", stage=DatasetStage.MODEL)
passport = store.get_passport(dataset_id=dataset_id)
ds = store.get(passport=passport)
train_df = ds.data

2025-10-24 01:05:59.743 | DEBUG    | valuation.asset.dataset.base:load:338 - Dataset Dataset train_val of the model stage created on 2025-10-24 at 00:50 loaded.


## Check Date Differences and Number of Periods

In [3]:
# Check frequency of the 'ds' column in your train_df
# Calculate the difference between consecutive dates for a sample series
sample_id = train_df['unique_id'].iloc[0]
sample_series_dates = train_df[train_df['unique_id'] == sample_id]['ds'].sort_values()
date_diffs = sample_series_dates.diff().dropna()

print("Frequency check for a sample series:")
print(date_diffs.value_counts())

# Check for missing periods per series
completeness_check = train_df.groupby('unique_id')['ds'].agg(['min', 'max', 'count'])
expected_weeks_train = (train_df['ds'].max() - train_df['ds'].min()).days // 7 + 1 # Approximate for 6 years

print(f"\nExpected number of weeks in train_df (approx): {expected_weeks_train}")
print("\nChecking completeness per series:")
print(completeness_check.head())

# Find series that don't have the expected number of weeks
incomplete_series = completeness_check[completeness_check['count'] != expected_weeks_train]
if not incomplete_series.empty:
    print(f"\nFound {len(incomplete_series)} series with potentially missing periods:")
    print(incomplete_series.head())
else:
    print("\nAll series appear to have the expected number of periods.")

Frequency check for a sample series:
ds
7 days    312
Name: count, dtype: int64

Expected number of weeks in train_df (approx): 313

Checking completeness per series:
                            min        max  count
unique_id                                        
100_analgesics       1990-01-03 1995-12-27    313
100_bath soap        1990-01-03 1995-12-27    313
100_bathroom tissues 1990-01-03 1995-12-27    313
100_beer             1990-01-03 1995-12-27    313
100_bottled juices   1990-01-03 1995-12-27    313

All series appear to have the expected number of periods.


## Series Length Statistics

In [4]:
# How much history do you have per series?
series_length = train_df.groupby('unique_id')['ds'].agg(['min', 'max', 'count'])
series_length['weeks'] = (series_length['max'] - series_length['min']).dt.days / 7

print("Series length distribution:")
print(series_length['weeks'].describe())
print(f"\nSeries with < 260 weeks: {(series_length['weeks'] < 260).sum():,}")
print(f"Series with >= 260 weeks: {(series_length['weeks'] >= 260).sum():,}")

Series length distribution:
count    2600.0
mean      312.0
std         0.0
min       312.0
25%       312.0
50%       312.0
75%       312.0
max       312.0
Name: weeks, dtype: float64

Series with < 260 weeks: 0
Series with >= 260 weeks: 2,600


## Diagnosing Time Series Horizon Issues

In [5]:
# Check 1: Verify your data's actual frequency
print("Checking data frequency...")
for uid in train_df['unique_id'].unique()[:5]:  # Check first 5 series
    ts = train_df[train_df['unique_id'] == uid].sort_values('ds')
    date_diffs = ts['ds'].diff().dt.days.value_counts()
    print(f"\n{uid}:")
    print(date_diffs.head())

# Check 2: What day of week are your dates?
print("\nDay of week distribution:")
print(train_df['ds'].dt.day_name().value_counts())

# Check 3: Are there gaps?
print("\nChecking for gaps...")
for uid in train_df['unique_id'].unique()[:10]:
    ts = train_df[train_df['unique_id'] == uid].sort_values('ds')
    expected_periods = (ts['ds'].max() - ts['ds'].min()).days // 7 + 1
    actual_periods = len(ts)
    if expected_periods != actual_periods:
        print(f"{uid}: expected {expected_periods}, got {actual_periods}")

Checking data frequency...

100_analgesics:
ds
7.0    312
Name: count, dtype: int64

100_bath soap:
ds
7.0    312
Name: count, dtype: int64

100_bathroom tissues:
ds
7.0    312
Name: count, dtype: int64

100_beer:
ds
7.0    312
Name: count, dtype: int64

100_bottled juices:
ds
7.0    312
Name: count, dtype: int64

Day of week distribution:
ds
Wednesday    813800
Name: count, dtype: int64

Checking for gaps...


## 

## Distribution of Revenue by Store and Category

In [8]:
# Assuming df_panel is your dense DataFrame with columns:
# ['store', 'category', 'y'] where 'y' is the revenue (0-filled)

from importlib.metadata import distribution


def analyze_revenue_distribution(df: pd.DataFrame) -> pd.DataFrame:
    """
    Performs a deep distribution analysis of the revenue (y) across all
    specified hierarchy levels.
    """
    
    # Split the unique_id into store and category
    df['store'] = df['unique_id'].apply(lambda s: s.split('_')[0])
    df['category'] = df['unique_id'].apply(lambda s: s.split('_')[1])
    
    # Define the hierarchy levels to analyze
    levels = [
        ['store', 'category'],  # STORE_CATEGORY (bottom level)
        ['store'],              # STORE
        ['category'],           # CATEGORY
        []                      # Overall (All Hierarchy Levels)
    ]
    
    analysis_results = []

    for level_cols in levels:
        
        # Determine the name for the current level
        if not level_cols:
            level_name = "OVERALL"
        else:
            level_name = "_".join(col.upper() for col in level_cols)
            
        print(f"--- Analyzing Level: {level_name} ---")

        # 1. Aggregate the Data
        if level_cols:
            # Group by the current level and calculate descriptive stats for revenue ('y')
            grouped_stats = df.groupby(level_cols)['y'].agg(
                mean_revenue='mean',
                median_revenue='median',
                std_revenue='std',
                min_revenue='min',
                max_revenue='max',
                p90_revenue=lambda x: x.quantile(0.90),
                p95_revenue=lambda x: x.quantile(0.95),
                total_series='size', # Number of time-steps for this group
                true_zeros=lambda x: (x == 0).sum() # Count of weeks with zero sales
            ).reset_index()
            
            # Now, calculate the number of unique series in this level
            n_unique_series = grouped_stats.shape[0]
            
            # Calculate the average of the metrics across all series in this level
            level_summary = grouped_stats[[
                'mean_revenue', 'median_revenue', 'std_revenue', 
                'p90_revenue', 'p95_revenue', 'total_series', 'true_zeros'
            ]].mean().to_dict()
            
            # Add the unique series count
            level_summary['n_unique_series'] = n_unique_series
            
            # Calculate total non-zero observations for sparsity check
            level_summary['total_observations'] = grouped_stats['total_series'].sum()
            level_summary['total_zeros'] = grouped_stats['true_zeros'].sum()
            
        else:
            # For OVERALL level, use the entire 'y' column
            level_summary = {
                'n_unique_series': df.groupby(['store', 'category']).ngroups, # Total bottom-level series
                'mean_revenue': df['y'].mean(),
                'median_revenue': df['y'].median(),
                'std_revenue': df['y'].std(),
                'min_revenue': df['y'].min(),
                'max_revenue': df['y'].max(),
                'p90_revenue': df['y'].quantile(0.90),
                'p95_revenue': df['y'].quantile(0.95),
                'total_observations': len(df),
                'total_zeros': (df['y'] == 0).sum(),
            }
        
        # 2. Calculate Derived Metrics (Sparsity, Volatility)
        level_summary['level'] = level_name
        
        # Sparsity: Percentage of zero-sale weeks
        level_summary['sparsity_percent'] = (level_summary['total_zeros'] / level_summary['total_observations']) * 100
        
        # Coefficient of Variation (CV) as a measure of relative volatility
        # Note: Using the mean of std_revenue divided by the mean of mean_revenue for grouped levels
        if level_name != "OVERALL" and level_summary['mean_revenue'] != 0:
            # Calculate average CV for the groups
            grouped_stats['cv'] = grouped_stats['std_revenue'] / grouped_stats['mean_revenue']
            level_summary['avg_cv'] = grouped_stats['cv'].mean()
        elif level_name == "OVERALL" and level_summary['mean_revenue'] != 0:
             level_summary['avg_cv'] = level_summary['std_revenue'] / level_summary['mean_revenue']
        else:
             level_summary['avg_cv'] = np.nan
        
        analysis_results.append(level_summary)

    # Convert results list to a final summary DataFrame
    summary_df = pd.DataFrame(analysis_results).set_index('level')
    
    # Select and format final columns for clean output
    final_cols = [
        'n_unique_series', 'total_observations', 'sparsity_percent', 'avg_cv',
        'mean_revenue', 'median_revenue', 'std_revenue', 'p90_revenue', 'p95_revenue'
    ]
    
    # Ensure all columns exist, handle missing ones (like when level is OVERALL)
    for col in final_cols:
        if col not in summary_df.columns:
            summary_df[col] = np.nan
            
    return summary_df[final_cols].round(2)



distribution_summary = analyze_revenue_distribution(train_df)
print("\n==================================================================================")
print("REVENUE DISTRIBUTION SUMMARY BY HIERARCHY LEVEL")
print("==================================================================================")
print(distribution_summary)

--- Analyzing Level: STORE_CATEGORY ---
--- Analyzing Level: STORE ---
--- Analyzing Level: CATEGORY ---
--- Analyzing Level: OVERALL ---

REVENUE DISTRIBUTION SUMMARY BY HIERARCHY LEVEL
                n_unique_series  total_observations  sparsity_percent  avg_cv  \
level                                                                           
STORE_CATEGORY             2600              813800             10.71     NaN   
STORE                        93              813800             10.71     NaN   
CATEGORY                     28              813800             10.71    0.61   
OVERALL                    2600              813800             10.71    1.36   

                mean_revenue  median_revenue  std_revenue  p90_revenue  \
level                                                                    
STORE_CATEGORY       7550.86         7386.90      2619.32     10402.15   
STORE                7550.46         4333.25      9135.09     18388.65   
CATEGORY             7555.80  